### Switching probability in first 100 trials
- Unstructured sessions showed higher alpha_chosen and lower alpha_perseverance compared to structured sessions, suggesting animals are more likely to switch in the unstructured sessions.

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    switch_prob = mab.get_switch_prob()
    print(f"Switch prob: {switch_prob}")

    df = pd.DataFrame(
        {
            "switch_prob": [switch_prob],
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )

    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_100trials")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_100trials

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="grp", y="switch_prob", hue="grp", ax=ax)
sns.stripplot(
    palette=["gray", "gray"],
    edgecolor="white",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    # errorbar="se",
    **plot_kw,
)
sns.barplot(
    palette="husl",
    # edgecolor="0.5",
    # facecolor=(0, 0, 0, 0),
    alpha=0.4,
    errorbar="se",
    **plot_kw,
)


orders = ["unstruc", "struc"]
pairs = [(("unstruc"), ("struc"))]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()

ax.set_title("Switch probability")
# ax.legend("")

### Switching probability as a function of trials within session
- Check if switch probability curves for unstructured and structured sessions have same decay and tail

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d


exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    switch_prob = mab.get_switch_prob_by_trial()
    switch_prob_smooth = gaussian_filter1d(switch_prob, sigma=2)

    df = pd.DataFrame(
        {
            "trial_id": np.arange(len(switch_prob)) + 1,
            "switch_prob": switch_prob,
            "switch_prob_smooth": switch_prob_smooth,
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )

    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_by_trial_100trials")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_by_trial_100trials

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="trial_id", y="switch_prob", hue="grp", ax=ax)
sns.lineplot(
    palette="husl",
    errorbar="se",
    err_kws={"edgecolor": None},
    **plot_kw,
)

# orders = ["unstruc", "struc"]
# pairs = [(("unstruc"), ("struc"))]
# annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
# annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()

ax.set_title("Switch probability")
# ax.legend("")
ax.set_xlim(1, 100)